In [2]:
import pandas as pd
import datetime
from datetime import datetime
import matplotlib as plt
import scipy
from scipy.stats import chi2_contingency

In [3]:
#функция будет принимать в себя дату, количество дней, на которое нужно рассчитать retantion, два датасета 
# с начальными данными. 
# Будет проводить обработку данных, считать retantion rate на заданное количество дней для этой даты 
# и для следующих дней после этой даты до истечения срока, который и задается количеством дней и выводить
# таблицу с retantion для анализа
def retantion_rate (d1, n, sr, sa):
# d1 - дата
# n - количество дней, на которое нужно рассчитать retantion
# sr - датасет с данными по регистрации
# sa - датасет с данными по авторизации
    demo_registration = pd.read_csv(sr)
    demo_authorization = pd.read_csv(sa)
    #подготовка данных
    registration = pd.DataFrame(columns=['registration_date', 'uid'])
    registration[['registration_date', 'uid']] = demo_registration['reg_ts;uid'].str.split(';', expand=True)
    authorization = pd.DataFrame(columns=['authorization_date', 'uid'])
    authorization[['authorization_date', 'uid']] = demo_authorization['auth_ts;uid'].str.split(';', expand=True)
    registration['registration_date'] = pd.to_datetime(registration["registration_date"], unit="s")
    registration['registration_date'] = pd.to_datetime(registration['registration_date']).dt.date
    registration['registration_date'] = pd.to_datetime(registration['registration_date'])
    authorization['authorization_date'] = pd.to_datetime(authorization['authorization_date'], unit="s")
    authorization['authorization_date'] = pd.to_datetime(authorization['authorization_date']).dt.date
    authorization['authorization_date'] = pd.to_datetime(authorization['authorization_date'])
# данные готовы, можно считать retantion
# таблица с одной пустой колонкой, которая будет наполняться значениями и выводиться на экран для анализа
    main_freim = pd.DataFrame(columns=['дата'])
    for u in range(1, n + 1):
        # определение даты, от которой будет запускаться цикл
        date_for_work_in_this_itereition = datetime.strptime(d1, "%Y-%m-%d").date() + timedelta(days= u - 1 )
        # добаляю эту дату в ту самую итоговую таблицу в колонку 'дата'
        main_freim.loc[u, 'дата'] = date_for_work_in_this_itereition
        # добавление в эту талицу новых колонок, сначала они пустые, их названия - даты,
        #содержание -renention для этих дат, заполняться они будут в цикле for, который ниже
        main_freim[date_for_work_in_this_itereition] = pd.NA
        # для каждой следующей даты нужно рассчитывать retention на 1 день меньше, это и есть то количество дней
        number_of_days_we_need_retention = n + 1 - u
        # количество пользователей, зарегистрировавшихся в этот день
        number_of_registrations = registration.query('registration_date == @date_for_work_in_this_itereition').uid.nunique()
        # список id этих пользователей
        df_of_users_registrated_this_day = registration.query('registration_date == @date_for_work_in_this_itereition')
        list_of_users_registrated_this_day = df_of_users_registrated_this_day['uid'].tolist()
        # датасет авторизаций только по этим пользователям
        authorization_who_registrated_date_for_work_in_this_itereition = authorization.query("uid in @list_of_users_registrated_this_day")
        # цикл для вычисления retention и добаления этих значений в таблицу
        for i in range(1, number_of_days_we_need_retention + 1):
            # определение даты, от которой будет запускаться цикл
            neded_date = date_for_work_in_this_itereition + timedelta(days = i - 1)
            # датасет авторизаций на определенное число только по тем самым пользователям (зарегистрировавшимся в 
            # определенный день(date_for_work_in_this_itereition)) 
            autorisations_this_date = authorization_who_registrated_date_for_work_in_this_itereition.query('authorization_date == @neded_date')
            # их количество
            number_of_autorisations_users = autorisations_this_date['uid'].nunique()
            # подсчет retention
            RR = round(number_of_autorisations_users / number_of_registrations * 100, 2)
            # добавление retention в таблицу
            main_freim.loc[u + i - 1, date_for_work_in_this_itereition] = RR
    print(main_freim)
# названия столбцов - даты регистрации, названия строк - даты, на которые показан retention пользователей, зарегистрировшихся
# в даты регистрации

In [5]:
retantion_rate('2019-09-23', 7, 'problem1-reg_data.csv', 'problem1-auth_data.csv')

         дата 2019-09-23 2019-09-24 2019-09-25 2019-09-26 2019-09-27  \
1  2019-09-23        100       <NA>       <NA>       <NA>       <NA>   
2  2019-09-24       1.56        100       <NA>       <NA>       <NA>   
3  2019-09-25       3.22       1.77        100       <NA>       <NA>   
4  2019-09-26       3.89       3.33       2.77        100       <NA>   
5  2019-09-27       5.22       4.21       4.42       3.09        100   
6  2019-09-28       5.22       3.33       4.42       3.76       1.65   
7  2019-09-29       7.33       4.55       5.75       4.86       3.53   

  2019-09-28 2019-09-29  
1       <NA>       <NA>  
2       <NA>       <NA>  
3       <NA>       <NA>  
4       <NA>       <NA>  
5       <NA>       <NA>  
6        100       <NA>  
7       1.65        100  
